# 1/ Import Packages

In [70]:
#!pip install pandas
!pip install xlsxwriter

     -------------------------------------- 159.9/159.9 kB 3.2 MB/s eta 0:00:00


In [64]:
import pandas as pd
import re
from datetime import datetime
import os

# 2/ Import Data

In [9]:
def import_excel_sheets(file_path):
    """
    Import all sheets from an Excel file as DataFrames.

    Parameters:
        file_path (str): Path to the Excel file.

    Returns:
        dict: A dictionary where keys are sheet names and values are DataFrames.
    """
    # Read all sheets into a dictionary of DataFrames
    sheets_dict = pd.read_excel(file_path, sheet_name=None)

    # Display sheet names and import each sheet as a DataFrame dynamically
    sheet_dataframes = {}
    for sheet_name, dataframe in sheets_dict.items():
        sheet_dataframes[sheet_name] = dataframe
        print(f"Imported sheet: {sheet_name}")

    return sheet_dataframes

In [11]:
dataframes = import_excel_sheets('input/cxo_data.xlsx')

Imported sheet: CEO
Imported sheet: COO
Imported sheet: CFO
Imported sheet: CMO
Imported sheet: CTO
Imported sheet: CSO
Imported sheet: CPO
Imported sheet: CCO
Imported sheet: CPO_HR
Imported sheet: CSO_Sustainability
Imported sheet: CDO


In [15]:
dataframes['CEO']

,Quarter,Overall Revenue Growth,Market Share,Employee Engagement Score,Customer Satisfaction Score,Revenue,Cost,Profit
0,2022-Q1,0.05,0.15,0.70,0.85,105000,70000,35000
1,2022-Q2,0.07,0.16,0.72,0.87,112350,73500,38850
2,2022-Q3,0.06,0.17,0.74,0.89,119091,77000,42091
3,2022-Q4,0.08,0.18,0.76,0.90,128618,80500,48118
4,2023-Q1,0.10,0.19,0.78,0.91,141480,84000,57480
5,2023-Q2,0.09,0.20,0.80,0.92,154213,87500,66713
6,2023-Q3,0.11,0.21,0.82,0.93,171177,91000,80177
7,2023-Q4,0.12,0.22,0.84,0.94,191718,94500,97218
8,2024-Q1,0.13,0.23,0.86,0.95,216641,98000,118641
9,2024-Q2,0.14,0.24,0.88,0.96,246971,101500,145471


# 3/ Clean Column Names

In [19]:
def sanitize_column_names(df):
    """
    Takes a DataFrame and sanitizes column names to be lowercase, without special characters,
    and replaces spaces with underscores. The changes are made directly to the DataFrame.

    Parameters:
        df (pd.DataFrame): Input DataFrame

    Returns:
        pd.DataFrame: DataFrame with sanitized column names
    """
    def sanitize(name):
        # Replace spaces with underscores
        name = name.replace(" ", "_")
        # Remove special characters except underscores
        name = re.sub(r'[^a-zA-Z0-9_]', '', name)
        # Convert to lowercase
        return name.lower()

    df.columns = [sanitize(col) for col in df.columns]
    return df

# 4/ Clean All DataFrames

In [25]:
def process_dataframes(dataframes):
    """
    Apply the sanitize_column_names function to all DataFrames in a dictionary.

    Parameters:
        dataframes (dict): Dictionary of DataFrames.

    Returns:
        dict: Dictionary with processed DataFrames.
    """
    processed_dataframes = {}
    for sheet_name, df in dataframes.items():
        print(f"Processing sheet: {sheet_name}")
        processed_dataframes[sheet_name] = sanitize_column_names(df)
    return processed_dataframes

In [27]:
processed_dataframes = process_dataframes(dataframes)

Processing sheet: CEO
Processing sheet: COO
Processing sheet: CFO
Processing sheet: CMO
Processing sheet: CTO
Processing sheet: CSO
Processing sheet: CPO
Processing sheet: CCO
Processing sheet: CPO_HR
Processing sheet: CSO_Sustainability
Processing sheet: CDO


In [32]:
processed_dataframes['CEO']

,quarter,overall_revenue_growth,market_share,employee_engagement_score,customer_satisfaction_score,revenue,cost,profit
0,2022-Q1,0.05,0.15,0.70,0.85,105000,70000,35000
1,2022-Q2,0.07,0.16,0.72,0.87,112350,73500,38850
2,2022-Q3,0.06,0.17,0.74,0.89,119091,77000,42091
3,2022-Q4,0.08,0.18,0.76,0.90,128618,80500,48118
4,2023-Q1,0.10,0.19,0.78,0.91,141480,84000,57480
5,2023-Q2,0.09,0.20,0.80,0.92,154213,87500,66713
6,2023-Q3,0.11,0.21,0.82,0.93,171177,91000,80177
7,2023-Q4,0.12,0.22,0.84,0.94,191718,94500,97218
8,2024-Q1,0.13,0.23,0.86,0.95,216641,98000,118641
9,2024-Q2,0.14,0.24,0.88,0.96,246971,101500,145471


In [34]:
processed_dataframes['CEO'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   quarter                      10 non-null     object 
 1   overall_revenue_growth       10 non-null     float64
 2   market_share                 10 non-null     float64
 3   employee_engagement_score    10 non-null     float64
 4   customer_satisfaction_score  10 non-null     float64
 5   revenue                      10 non-null     int64  
 6   cost                         10 non-null     int64  
 7   profit                       10 non-null     int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 768.0+ bytes


# 5/ Create Summary

In [37]:
def create_reports(processed_dataframes):
    """
    Create a report for each DataFrame in the processed_dataframes dictionary.

    Parameters:
        processed_dataframes (dict): Dictionary of processed DataFrames.

    Returns:
        dict: Dictionary where keys are sheet names and values are the generated reports (DataFrames).
    """
    reports = {}
    for sheet_name, df in processed_dataframes.items():
        try:
            print(f"Creating report for sheet: {sheet_name}")
            df_report = df.groupby('quarter').sum().tail().transpose()
            reports[sheet_name] = df_report
        except Exception as e:
            print(f"Error creating report for sheet {sheet_name}: {e}")
    return reports

In [39]:
reports = create_reports(processed_dataframes)

Creating report for sheet: CEO
Creating report for sheet: COO
Creating report for sheet: CFO
Creating report for sheet: CMO
Creating report for sheet: CTO
Creating report for sheet: CSO
Creating report for sheet: CPO
Creating report for sheet: CCO
Creating report for sheet: CPO_HR
Creating report for sheet: CSO_Sustainability
Creating report for sheet: CDO


In [41]:
reports['CEO']

quarter,2023-Q2,2023-Q3,2023-Q4,2024-Q1,2024-Q2
overall_revenue_growth,0.09,0.11,0.12,0.13,0.14
market_share,0.20,0.21,0.22,0.23,0.24
employee_engagement_score,0.80,0.82,0.84,0.86,0.88
customer_satisfaction_score,0.92,0.93,0.94,0.95,0.96
revenue,154213.00,171177.00,191718.00,216641.00,246971.00
cost,87500.00,91000.00,94500.00,98000.00,101500.00
profit,66713.00,80177.00,97218.00,118641.00,145471.00


# 5/ Export Report

In [45]:
def export_dataframe_with_timestamp(df, filename_prefix="ceo_report"):
    """
    Exports the given DataFrame to a CSV file in the output folder with a timestamped filename.

    Parameters:
        df (pd.DataFrame): The DataFrame to export.
        filename_prefix (str): The prefix for the output file name.

    Returns:
        None
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = f'output/{timestamp}_{filename_prefix}.csv'
    df.to_csv(output_path)
    print(f"File exported to {output_path}")

In [55]:
def export_all_reports(reports):
    """
    Export all reports in the dictionary to timestamped CSV files.

    Parameters:
        reports (dict): Dictionary of DataFrames to export.
    """
    for sheet_name, df_report in reports.items():
        try:
            export_dataframe_with_timestamp(df_report, sheet_name)
        except Exception as e:
            print(f"Error exporting report for sheet {sheet_name}: {e}")

In [57]:
export_all_reports(reports)

File exported to output/20250105_165625_CEO.csv
File exported to output/20250105_165625_COO.csv
File exported to output/20250105_165625_CFO.csv
File exported to output/20250105_165625_CMO.csv
File exported to output/20250105_165625_CTO.csv
File exported to output/20250105_165625_CSO.csv
File exported to output/20250105_165625_CPO.csv
File exported to output/20250105_165625_CCO.csv
File exported to output/20250105_165625_CPO_HR.csv
File exported to output/20250105_165625_CSO_Sustainability.csv
File exported to output/20250105_165625_CDO.csv


In [76]:
def export_to_excel(reports, output_file="output/all_sheets.xlsx"):
    """
    Export all reports to a single Excel file.

    Parameters:
        reports (dict): Dictionary of DataFrames to export.
        output_file (str): Path to the output Excel file.
    """
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
        for sheet_name, df_report in reports.items():
            try:
                df_report.to_excel(writer, sheet_name=sheet_name)
                print(f"Added sheet: {sheet_name} to {output_file}")
            except Exception as e:
                print(f"Error exporting sheet {sheet_name} to Excel: {e}")

In [78]:
export_to_excel(reports)

Added sheet: CEO to output/all_sheets.xlsx
Added sheet: COO to output/all_sheets.xlsx
Added sheet: CFO to output/all_sheets.xlsx
Added sheet: CMO to output/all_sheets.xlsx
Added sheet: CTO to output/all_sheets.xlsx
Added sheet: CSO to output/all_sheets.xlsx
Added sheet: CPO to output/all_sheets.xlsx
Added sheet: CCO to output/all_sheets.xlsx
Added sheet: CPO_HR to output/all_sheets.xlsx
Added sheet: CSO_Sustainability to output/all_sheets.xlsx
Added sheet: CDO to output/all_sheets.xlsx
